In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In C:\Users\Flayiva_01\anaconda3\envs\pytorchenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\Flayiva_01\anaconda3\envs\pytorchenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\Flayiva_01\anaconda3\envs\pytorchenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
# from torchsummary import summary

from scipy import ndimage
import numpy as np
import PIL
import shutil
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

from utilities.common_utils import *
from utilities.landmark_utils import *
from utilities.plotting import *
#from models import ResUNet, UNet


## Get dataset
We will use the 'A Grand Challenge for Automated Detection of Anatomical Landmarks and Analysis for Diagnosis in Cephalometric X-ray Image' dataset, see  http://www-o.ntust.edu.tw/~cweiwang/ISBI2015/challenge1/index.html

Download 'A Grand Challenge for Automated Detection of Anatomical Landmarks and Analysis for Diagnosis in Cephalometric X-ray Image
data, see  http://www-o.ntust.edu.tw/~cweiwang/ISBI2015/challenge1/index.html

In [3]:
path = Path('./data'); path.mkdir(exist_ok=True)

In [4]:
path

WindowsPath('data')

### Download

In [ ]:
URL = 'https://ndownloader.figshare.com/articles/3471833?private_link=37ec464af8e81ae6ebbf'
!cd {path}
!curl -o {path}/dataset.zip {URL}

### Unpack

In [ ]:
!unzip -d {path} {path}/dataset.zip

In [ ]:
!unar -q -f -o {path} {path}/AnnotationsByMD.rar
!unar -q -f -o {path} {path}/EvaluationCode.rar
!unar -q -f -o {path} {path}/RawImage.rar

### Create subsampled training and test datasets

In [5]:
test1_orig_path = path/'RawImage/Test1Data'
test2_orig_path = path/'RawImage/Test2Data'
train_orig_path =  path/'RawImage/TrainingData'

IMAGE_SIZE = 256

In [7]:
test1_orig_path

WindowsPath('data/RawImage/Test1Data')

In [8]:
def resize(fpath, img_size, resized_dir):
    PIL.Image.open(fpath).resize((img_size, img_size)).save(resized_dir/fpath.name)

Resize test images

In [16]:
test_path = path/f'images/raw/{IMAGE_SIZE}/test1'; test_path.mkdir(parents=True, exist_ok=True)
l_resize = lambda fpath : resize(fpath, IMAGE_SIZE, test_path)
with ThreadPoolExecutor(16) as e: e.map(l_resize, list_files(test1_orig_path))

In [5]:
test_path = path/f'images/raw/{IMAGE_SIZE}/test2'; test_path.mkdir(parents=True, exist_ok=True)
l_resize = lambda fpath : resize(fpath, IMAGE_SIZE, test_path)
with ThreadPoolExecutor(16) as e: e.map(l_resize, list_files(test2_orig_path))

In [6]:
test_path

WindowsPath('data/images/raw/256/test2')

Resize train images

In [19]:
train_path = path/f'images/raw/{IMAGE_SIZE}/train'; train_path.mkdir(exist_ok=True) 
l_resize = lambda fpath : resize(fpath, IMAGE_SIZE, train_path)
with ThreadPoolExecutor(16) as e: e.map(l_resize, list_files(train_orig_path))

In [23]:
train_path

PosixPath('data/images/256/train')